# Coding Challenge Huk-Coburg

## Business Understanding

* Given is a dataset of french car insurances with raised damage claims
* Time budget 5h
* predict the yearly damage claims for an insurance holder. 

## Data Understanding

In [ ]:
import pandas as pd 
import arff

In [ ]:
# Load features and 
data_freq = arff.load('data/freMTPL2freq.arff')
df_freq = pd.DataFrame(data_freq, columns=["IDpol", "ClaimNb", "Exposure", "Area", "VehPower", 
                                           "VehAge","DrivAge", "BonusMalus", "VehBrand", "VehGas", 
                                           "Density", "Region"])

In [ ]:
data_sev = arff.load('freMTPL2sev.arff')
df_sev = pd.DataFrame(data_sev, columns=["IDpol", "ClaimAmount"])